LSE Data Science Institute | ME204 (2024) | Final Project

# 🎼 Building Albums and Track Audio Features Databases from Spotify API 

<span style="display: inline-block; padding: 0 10px; font-size: 1.15em;line-height: 1.5em; white-space: nowrap; border: 1px solid #E69F25; border-radius: .5em; color: #fcfcfc; background-color: #E69F25; vertical-align: middle;font-weight: 600 !important;">Data Base Building NOTEBOOK</span>

**DATE:** 26 July 2024

**AUTHOR:** [David Cho](https://github.com/LSE-ME204/me204-2024-project-Chodav?tab=readme-ov-file)

-----


# ⚙️ **Setup**

**IMPORTS:**

In [1]:
import os
import sqlite3

import pandas as pd

import re
import json

# **🏭 1. Converting jsons into pd.DataFrames**

In [2]:
def find_files_with_string(directory, search_string):
    """
    Find all files with a certain string.

    Parameters:
    directory (str): The directory where the files are stored.
    search_string (str): The string in a file name to be found.

    Returns:
    list: Collection of all the file paths containing the search_string.
    """

    # List to store the paths of matching files
    matching_files = []

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if search_string in file:
                # Append the full path of the matching file
                full_path = os.path.join(root, file)
                matching_files.append(full_path)
    
    return matching_files

## **Albums**

In [3]:
directory = '../data/raw/'
search_string = 'albums'

album_paths = find_files_with_string(directory, search_string)

In [4]:
def convert_albums_json(file_path, genre, artist_name):
    """
    Convert the album json into a pandas df.

    Parameters:
    file_path (str): The path of the file.
    genre (str): The genre of the artist.
    artist_name (str): The name of the artist.

    Returns:
    pd.DataFrame : A data frame of an artist's albums information.
    """

    df = pd.read_json(file_path)

    df['genres'] = genre 
    df['artists'] = artist_name
    df['available_markets'] = df['available_markets'].apply(lambda x: len(x))
    df.drop(columns=['album_type', 'copyrights', 'external_ids', 'external_urls', 'href', 'images', 'label', 'tracks', 'type', 'uri'], inplace=True)
    df.rename(columns={'artists': 'artist', 'genres': 'genre'}, inplace=True)

    return df

def format_artist_name(file_path, mode):
    """
    Extract the artist name from the file name.

    Parameters:
    file_path (str): The path of the file.
    mode (str): The type of collection the json holds.

    Returns:
    str : The name of the artist.
    """

    if mode == 'album':
        # Extract the artist name from the file path
        artist_name_match = re.search(r'/raw/([^/]+)_albums\.json$', file_path)
        
        if artist_name_match:
            artist_name = artist_name_match.group(1)
            # Replace underscores with spaces and capitalize properly
            artist_name = artist_name.replace('_', ' ').title()
            return artist_name
        else:
            return 'Unknown Artist'
    
    elif mode == 'audio5':
        # Get the base name of the file without the extension
        base_name = os.path.basename(file_path)
        # Remove known suffix parts from the base name
        name_part = base_name.replace('_5audio_features.json', '').replace('_albums.json', '')
        # Replace underscores with spaces and capitalize each word
        artist_name = ' '.join(word.capitalize() for word in name_part.split('_'))

        return artist_name
    
    elif mode == 'track5':
        # Get the base name of the file without the extension
        base_name = os.path.basename(file_path)
        # Remove known suffix parts from the base name
        name_part = base_name.replace('_5track_info.json', '').replace('_albums.json', '')
        # Replace underscores with spaces and capitalize each word
        artist_name = ' '.join(word.capitalize() for word in name_part.split('_'))

        return artist_name

In [ ]:
classical_artists = ['London Symphony Orchestra', 'Johann Sebastian Bach', 'Ludwig Van Beethoven', 'Wolfgang Amadeus Mozart',
                    'Frederic Chopin', 'Pyotr Ilyich Tchaikovsky', 'Antonio Vivaldi', 'Claude Debussy', 'Erik Satie', 'Camille Saint-Saens']

jazz_artists = ['Ella Fitzgerald', 'Stan Getz', 'Count Basie', 'Billie Holiday', 'Chet Baker', 'Miles Davis', 'John Coltrane',
                'The Dave Brubeck Quartet', 'Duke Ellington', 'Kamasi Washington']

indie_pop_artists = ['Chappell Roan', 'Clairo', 'Beabadoobee', 'Girl In Red', 'Phoebe Bridgers', 'Suki Waterhouse', 'Sufjan Stevens',
                    'Lizzy Mcalpine', 'Maggie Rogers', 'Beach Bunny']

gen_z_artists = ['Stephen Sanchez', 'David Kushner', 'Em Beihold', 'Laufey', 'Alex Warren', 'Matt Hansen', 'Lauren Spencer Smith',
                'Clinton Kane', 'Jax', 'Anson Seabra']

def get_artist_genre(artist_name):
    """
    Return the genre associated with an artist.

    Parameters:
    artist_name (str): The name of an artist.

    Returns:
    str : The genre associated with the artist.
    """

    if artist_name in classical_artists:
        return 'Classical'
    elif artist_name in jazz_artists:
        return 'Jazz'
    elif artist_name in indie_pop_artists:
        return 'Indie Pop'
    elif artist_name in gen_z_artists:
        return 'Gen Z'
    else:
        return 'Unknown Genre'

In [5]:
albums_df = pd.DataFrame()

for album_path in album_paths:
    artist_name = format_artist_name(album_path, 'album')
    genre = get_artist_genre(artist_name)
    album_df = convert_albums_json(album_path, genre, artist_name)
    albums_df = pd.concat([albums_df, album_df], axis=0, ignore_index=True)

# reconvert names
albums_df.loc[albums_df['artist'] == 'Girl In Red', 'artist'] = 'girl in red'
albums_df.loc[albums_df['artist'] == 'Beabadoobee', 'artist'] = 'beabadoobee'
albums_df.loc[albums_df['artist'] == 'Camille Saint-Saens', 'artist'] = 'Camille Saint-Saëns'
albums_df.loc[albums_df['artist'] == 'Frederic Chopin', 'artist'] = 'Frédéric Chopin'
albums_df.loc[albums_df['artist'] == 'Lizzy Mcalpine', 'artist'] = 'Lizzy McAlpine'

# Clean release dates
albums_df['release_year'] = albums_df['release_date'].apply(lambda x: int(x[:4]))
albums_df = albums_df.drop(columns=['release_date', 'release_date_precision'])


## **Artists Info**

In [7]:
directory = '../data/raw/'
search_string = 'info'

info_paths = find_files_with_string(directory, search_string)
info_paths = [path for path in info_paths if '5' not in path]

In [8]:
def convert_info_jsons(file_path):
    """
    Read an info json into a list.

    Parameters:
    file_path (str): The path of the file.

    Returns:
    list : A json nested as a list of the artist information.
    """

    with open(file_path, 'r') as file:
        data = json.load(file)

    artist_data = {
        'name' : data['name'],
        'popularity' : data['popularity'],
        'followers' : data['followers']['total'],
        'id' : data['id'],
        'genres' : data['genres']
    }

    return [artist_data]


In [9]:
infos_df = pd.DataFrame()

for file_path in info_paths:
    info_df = pd.DataFrame(convert_info_jsons(file_path))
    infos_df = pd.concat([infos_df, info_df], axis=0, ignore_index=True)

infos_df.rename(columns={'name': 'artist'}, inplace=True)

## **Audio Features (recent 5 albums)**

In [35]:
def convert_jsons(file_path):
    """
    Convert a json into a pandas df.

    Parameters:
    file_path (str): The path of the file.

    Returns:
    pd.DataFrame : The json converted as a df.
    """

    df = pd.read_json(file_path)
    return df

In [15]:
directory = '../data/raw/'
search_string = '5audio'

audio_paths = find_files_with_string(directory, search_string)

['../data/raw/rachmaninoff_5audio_features.json', '../data/raw/adele_5audio_features.json', '../data/raw/billie_holiday_5audio_features.json', '../data/raw/ella_fitzgerald_5audio_features.json', '../data/raw/laufey_5audio_features.json', '../data/raw/frederic_chopin_5audio_features.json', '../data/raw/chet_baker_5audio_features.json', '../data/raw/mendelssohn_5audio_features.json', '../data/raw/taylor_swift_5audio_features.json', '../data/raw/nora_jones_5audio_features.json', '../data/raw/ravel_5audio_features.json']


In [38]:
audios5_df = pd.DataFrame()

for file_path in audio_paths:
    artist_name = format_artist_name(file_path, 'audio5')
    audio5_df = convert_jsons(file_path)
    audio5_df['artist'] = artist_name
    
    audios5_df = pd.concat([audios5_df, audio5_df], axis=0, ignore_index=True)

# Reconvert names
audios5_df.loc[audios5_df['artist'] == 'Frederic Chopin', 'artist'] = 'Frédéric Chopin'
audios5_df.loc[audios5_df['artist'] == 'Dvorak', 'artist'] = 'Antonín Dvořák'
audios5_df.loc[audios5_df['artist'] == 'Mendelssohn', 'artist'] = 'Felix Mendelssohn'
audios5_df.loc[audios5_df['artist'] == 'Rachmaninoff', 'artist'] = 'Sergei Rachmaninoff'
audios5_df.loc[audios5_df['artist'] == 'Ravel', 'artist'] = 'Maurice Ravel'
audios5_df.loc[audios5_df['artist'] == 'Scriabin', 'artist'] = 'Alexander Scriabin'
audios5_df.loc[audios5_df['artist'] == 'Shostakovich', 'artist'] = 'Dmitri Shostakovich'

# Clean duration
audios5_df['duration_s'] = audios5_df['duration_ms'].apply(lambda x: x // 1000)

# Drop 'duration_ms' column
audios5_df = audios5_df.drop(columns=['duration_ms'])

## **Track Info (recent 50)**

In [19]:
directory = '../data/raw/'
search_string = '5track'

audio_paths = find_files_with_string(directory, search_string)

['../data/raw/ella_fitzgerald_5track_info.json', '../data/raw/frederic_chopin_5track_info.json', '../data/raw/taylor_swift_5track_info.json', '../data/raw/chet_baker_5track_info.json', '../data/raw/adele_5track_info.json', '../data/raw/nora_jones_5track_info.json', '../data/raw/laufey_5track_info.json', '../data/raw/mendelssohn_5track_info.json', '../data/raw/ravel_5track_info.json', '../data/raw/rachmaninoff_5track_info.json', '../data/raw/billie_holiday_5track_info.json']


In [20]:
tracks5_df = pd.DataFrame()

for file_path in audio_paths:
    artist_name = format_artist_name(file_path, "track5")
    track5_df = convert_jsons(file_path)
    track5_df['artist'] = artist_name
    
    tracks5_df = pd.concat([tracks5_df, track5_df], axis=0, ignore_index=True)

tracks5_df.rename(columns={'track_id': 'id'}, inplace=True)

# Reconvert names
tracks5_df.loc[tracks5_df['artist'] == 'Frederic Chopin', 'artist'] = 'Frédéric Chopin'
tracks5_df.loc[tracks5_df['artist'] == 'Dvorak', 'artist'] = 'Antonín Dvořák'
tracks5_df.loc[tracks5_df['artist'] == 'Mendelssohn', 'artist'] = 'Felix Mendelssohn'
tracks5_df.loc[tracks5_df['artist'] == 'Rachmaninoff', 'artist'] = 'Sergei Rachmaninoff'
tracks5_df.loc[tracks5_df['artist'] == 'Ravel', 'artist'] = 'Maurice Ravel'
tracks5_df.loc[tracks5_df['artist'] == 'Scriabin', 'artist'] = 'Alexander Scriabin'
tracks5_df.loc[tracks5_df['artist'] == 'Shostakovich', 'artist'] = 'Dmitri Shostakovich'

# Clean duration
tracks5_df['duration_s'] = tracks5_df['duration_ms'].apply(lambda x: x // 1000)

# Drop 'duration_ms' column
tracks5_df = tracks5_df.drop(columns=['duration_ms'])

In [21]:
# Spotify's API association between an artist and their albums is not entirely reliable
# The following code makes sure that only tracks with the correct artist are kept
tracks5_df.reset_index(drop=True, inplace=True)
tracks5_df = tracks5_df[tracks5_df.apply(lambda row: row['artist'] in row['artists'], axis=1)]


# **👨‍💻 2. Setting up a SQLite database**

In [24]:
DATA_FOLDER = os.path.join('../data/clean/')

In [25]:
%load_ext sql
%config SqlMagic.autocommit=True # for engines that do not support autommit

In [26]:
# Create a SQLite database in the ../data/ directory if it does not already exist
conn = sqlite3.connect(os.path.join(DATA_FOLDER, './spotify.db'))

In [27]:
albums_df = albums_df.drop_duplicates()
albums_df.to_sql('albums', conn, if_exists='replace', index=False)

# Filtering out albums that have less than popularity rating of 10
conn.execute('''
DELETE FROM albums
WHERE popularity < 6;
''')

In [28]:
# Convert the 'genres' column to strings as SQLite cannot store lists
infos_df['genres'] = infos_df['genres'].apply(lambda x: ', '.join(x))

infos_df = infos_df.drop_duplicates()
infos_df.to_sql('artists', conn, if_exists='replace', index=False)

49

In [29]:
audios5_df = audios5_df.drop_duplicates()
audios5_df.to_sql('audio_features', conn, if_exists='replace', index=False)

2302

In [30]:
tracks5_df = tracks5_df.drop(columns=['artists'])
tracks5_df = tracks5_df.drop_duplicates()
tracks5_df.to_sql('tracks', conn, if_exists='replace', index=False)

474

# **✈️ 3. Export as CSV files**

In [32]:
albums_df = pd.read_sql('SELECT artist, available_markets, genre, name, popularity, total_tracks, release_year FROM albums', conn)

artists_df = pd.read_sql('SELECT artist, popularity, followers, genres FROM artists', conn)

cleaned_tracks_df = (
    pd.read_sql('SELECT id, track_name, popularity, artist FROM tracks', conn)
        .merge(pd.read_sql('SELECT id, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo FROM audio_features', conn),
               left_on='id',
               right_on='id',
               how='left')
)

In [34]:
# Save data tables as CSV's to be accessed in NB03
albums_df.to_csv('../data/clean/albums.csv', index=False)
artists_df.to_csv('../data/clean/artists.csv', index=False)
cleaned_tracks_df.to_csv('../data/clean/tracks.csv', index=False)